In [50]:
import os
import pandas as pd
import numpy as np
from uuid import uuid4
import pickle

In [51]:
n,m = 10000, 50
frame = pd.DataFrame(data=np.random.randn(n, m), columns=[str(uuid4()) for _ in range(0, m)])

## Attempts without Mongo

In [ ]:
%%timeit
frame.to_parquet("test.parquet", engine="pyarrow")
pd.read_parquet("test.parquet", engine="pyarrow")

In [ ]:
%%timeit
frame.to_pickle("test.pickle")
pd.read_pickle("test.pickle")

In [ ]:
for file in os.listdir():
    print(file, os.stat(file).st_size)

## Connecting to Mongo

In [ ]:
MONGODB_SETTINGS = {'db': 'mongo', 'host':'mongo'}

from mongoengine import *
connect(**MONGODB_SETTINGS)

## Using Parquet, Convert frame to parquet and back

In [ ]:
from antarctic.PandasFields import ParquetFrameField

class Maffay(Document):
    frame = ParquetFrameField()

In [ ]:
%%timeit
m=Maffay(frame=frame).save()
m.frame

## Using Pickle, Convert using Pickle and back

In [ ]:
from antarctic.PandasFields import PickleFrameField

class Falco(Document):
    frame = PickleFrameField()

In [ ]:
%%timeit
f = Falco(frame=frame).save()
f.frame

In [ ]:
from antarctic.PandasFields import PickleFrameField

class Falco(Document):
    frame = PickleFrameField()

In [ ]:
%%timeit
f = Falco(frame=frame).save()
f.frame